<a href="https://colab.research.google.com/github/onaka-yurusugi/dataset_random_copy/blob/main/%E7%94%BB%E5%83%8F%E8%A4%87%E8%A3%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import glob
import random
import pathlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageOps

In [ ]:
class CvOverlayImage(object):
    def __init__(self):
         pass

    @classmethod
    def overlay(
            cls,
            cv_background_image,
            cv_overlay_image,
            point,
    ):
        overlay_height, overlay_width = cv_overlay_image.shape[:2]

        # OpenCV形式の画像をPIL形式に変換(α値含む)
        # 背景画像
        cv_rgb_bg_image = cv2.cvtColor(cv_background_image, cv2.COLOR_BGR2RGB)
        pil_rgb_bg_image = Image.fromarray(cv_rgb_bg_image)
        pil_rgba_bg_image = pil_rgb_bg_image.convert('RGBA')
        # オーバーレイ画像
        #cv_rgb_ol_image = cv2.cvtColor(cv_overlay_image, cv2.COLOR_BGRA2RGBA)
        cv_rgb_ol_image = cv_overlay_image
        pil_rgb_ol_image = Image.fromarray(cv_rgb_ol_image)
        pil_rgba_ol_image = pil_rgb_ol_image.convert('RGBA')

        # composite()は同サイズ画像同士が必須のため、合成用画像を用意
        pil_rgba_bg_temp = Image.new('RGBA', pil_rgba_bg_image.size,
                                     (255, 255, 255, 0))
        # 座標を指定し重ね合わせる
        pil_rgba_bg_temp.paste(pil_rgba_ol_image, point, pil_rgba_ol_image)
        result_image = \
            Image.alpha_composite(pil_rgba_bg_image, pil_rgba_bg_temp)

        # OpenCV形式画像へ変換
        cv_bgr_result_image = cv2.cvtColor(
            np.asarray(result_image), cv2.COLOR_RGBA2BGRA)

        return cv_bgr_result_image

def select_color(color):
    mean = np.array(color).mean(axis=0)
    return (0,0,0,0) if mean <= 1 else color

def to_touka(img):
    w, h = img.size
    touka_img = Image.new('RGBA', (w, h))
    np.array([[touka_img.putpixel((x, y), select_color(img.getpixel((x,y)))) for x in range(w)] for y in range(h)])
    return touka_img

def random_shift(image, shifts):
    h, w = image.shape[:2]
    src = np.array([[0.0, 0.0],[0.0, 1.0],[1.0, 0.0]], np.float32)
    dest = src + shifts.reshape(1,-1).astype(np.float32)
    affine = cv2.getAffineTransform(src, dest)
    return cv2.warpAffine(image, affine, (w, h))
    
def expand(image, ratio):
    h, w = image.shape[:2]
    src = np.array([[0.0, 0.0],[0.0, 1.0],[1.0, 0.0]], np.float32)
    dest = src * ratio
    affine = cv2.getAffineTransform(src, dest)
    return cv2.warpAffine(image, affine, (2*w, 2*h), cv2.INTER_LANCZOS4) # 補間法も指定できる
  
def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

# 画像複製(ランダムシフト+拡大処理)

In [ ]:
# 複製元の画像フォルダのパス
files = sorted(glob.glob("/content/drive/My Drive/Colab Notebooks/tmp/img/in_x/*"))

# 出力先フォルダのパス
out_path = "/content/drive/My Drive/Colab Notebooks/tmp/img/out/"

# 複製する枚数
out_num = 1

# (i = 1 → 00001.pngから連番で保存)
i = 500

# シフト先の座標の下限,上限(pix)
shift_min = -10
shift_max = 0

# 拡大倍率 (1 : 2.2-2.5, 2 : 2.15-2.30 3 : 2.07-2.14)
ratio_min = 2.07
ratio_max = 2.14

In [ ]:
# 基点の座標
point = (0, 0)

for file in files:
   try:     
      image = cv2.imread(file)[:,:,::-1]
   except TypeError:
      print("TypeError:" + str(pathlib.Path(file)))
   for idx in range(out_num):
      zero_i = "{0:05d}".format(i)
      imheight, imwidth = image.shape[:2]      

      # 拡大処理 ※重くなる     
      ratio = random.uniform(ratio_min,ratio_max) # 拡大倍率 原寸は2.0
      # image2 = image
      image2 = expand(image,ratio)
      
      # 差分画像の作成(ランダムシフト)
      shift = np.float32([[random.randint(shift_min,shift_max),random.randint(shift_min,shift_max)]])
      converted = random_shift(image2,shift)

      tmp_img = to_touka(Image.fromarray(np.uint8(converted))) # 画像の黒色部分を透過する
      cv_background_image = image # 背景用
      cv_overlay_image = pil2cv(tmp_img)
      cv_overlay_image = cv2.resize(cv_overlay_image, (imwidth, imheight))  
      tmp_img = CvOverlayImage.overlay(cv_background_image, cv_overlay_image,point) # 元画像と差分画像の合成処理
      result_image = Image.fromarray(np.uint8(tmp_img))
      result_image.save(out_path + zero_i + ".png") # 保存
      # plt.title(zero_i + ".png")
      # plt.imshow(result_image)
      # plt.show()
      print(str(pathlib.Path(file)) + "　→　" + out_path + zero_i + ".png")
      i = i + 1
print("end")

/content/drive/My Drive/Colab Notebooks/tmp/img/in_x/00499.png　→　/content/drive/My Drive/Colab Notebooks/tmp/img/out/00500.png
end


# リサイズ/左右反転

In [ ]:
# 変換元画像フォルダのパス
files = sorted(glob.glob("/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/*"))

# 出力先フォルダのパス
out_path = "/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/"

# (i = 1 → 00001.pngから連番で保存)
i = 501

In [ ]:
for file in files:
  img = Image.open(file)
  try:
    # リサイズ
    # トリミングのサイズ(ピクセル)
new_size = 400
    img = crop_center(img, 256, 256)
    file_name = file.split('.')[0]  # ファイル名から拡張子を除いた部分を取り出す

        
    # リサイズした画像を連番(00001.png～)で保存
    zero_i = "{0:05d}".format(i)
    tmp_img.save(out_path + zero_i + ".png") # 保存
    print("resize completed:" + file + " → " + zero_i + ".png")
    
    '''

    # 左右反転
    img = (img[:, ::-1])
    #i += 1
    zero_i = "{0:05d}".format(i)
    tmp_img.save(out_path + zero_i + ".png") # 保存
    '''

    print(str(pathlib.Path(file)) + "　→　" + out_path + zero_i + ".png")    
    
    i += 1
  except:
    print("error")

print("COMPLETED!")

resize completed:/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/00501.png → 00501.png
/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/00501.png　→　/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/00501.png
resize completed:/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/saigou002.jpg → 00502.png
/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/saigou002.jpg　→　/content/drive/My Drive/Colab Notebooks/tmp/img/in_s/00502.png
COMPLETED!


# 連番リネーム

In [ ]:
# 変換元画像フォルダのパス
files = sorted(glob.glob("/content/drive/My Drive/Colab Notebooks/tmp/img/out/*"))

# 出力先フォルダのパス
out_path = "/content/drive/My Drive/Colab Notebooks/tmp/img/out/"

# (i = 1 → 00001.pngから連番で保存)
i = 6

In [ ]:
for file in files:
  img = cv2.imread(file)
  try:
    file_name = file.split('.jpg')[0]

    # 数値を五桁(00001)形式にそろえる
    zero_i = "{0:05d}".format(i)

    cv2.imwrite(zero_i + ".png",img)

    print("rename completed:" + file_name + "  →  " + zero_i + ".png")
    i += 1

  except:
    print("error")

print("COMPLETED!")

# 参考

完全に理解するアフィン変換: https://qiita.com/koshian2/items/c133e2e10c261b8646bf

python画像処理9 透過PNGに変換する: https://webcache.googleusercontent.com/search?q=cache:u93DuZuumhEJ:https://www.uosansatox.biz/entry/2017/12/07/124831+&cd=2&hl=ja&ct=clnk&gl=jp

【OpenCV】【Python】画像上に透過付き画像を重ねて描画: https://qiita.com/Kazuhito/items/ff4d24cd012e40653d0c